In [1]:
import os
from google.colab import drive

drive.mount('/content/drive')
os.chdir('drive/My Drive/Colab Notebooks/AI RESIDENCY/P3')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
## data
import numpy as np
import pandas as pd

## visualisations
import seaborn as sns
import matplotlib.pyplot as plt

## ignore warnings
import warnings
warnings.filterwarnings("ignore",module = "matplotlib\..*")






!pip install --upgrade --pre gensim

import gensim
from gensim.models import Word2Vec
from gensim.models.fasttext import FastText
import gensim.models

!pip install glove-python-binary
from glove import Corpus, Glove


import re
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet 
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [12]:
## Load data
data = pd.read_csv('potential-talents - Aspiring human resources - seeking human resources.csv')
data.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [13]:
data.shape

(104, 5)

In [14]:
len(data['id'].unique())

104

In [15]:
data.drop('fit',axis=1,inplace=True)

In [16]:
## check for duplicates
duplicates_check = data.copy()
duplicates_check.drop(columns=['id'],axis=1,inplace=True)
print(f'There are {duplicates_check.duplicated().sum()} duplicates.')

There are 51 duplicates.


In [17]:
data = data.set_index('id')
## drop duplicates
data.drop_duplicates(inplace=True)
print(f'There are {data.shape[0]} distinct entries.')

There are 53 distinct entries.


In [18]:
data.head()

,job_title,location,connection
id,,,
1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85
2,Native English Teacher at EPIK (English Progra...,Kanada,500+
3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44
4,People Development Coordinator at Ryan,"Denton, Texas",500+
5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+


In [19]:
data['job_title'].value_counts()

Aspiring Human Resources Professional                                                                                    2
2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional                 1
Lead Official at Western Illinois University                                                                             1
Senior Human Resources Business Partner at Heil Environmental                                                            1
Aspiring Human Resources Professional | An energetic and Team-Focused Leader                                             1
HR Manager at Endemol Shine North America                                                                                1
Human Resources professional for the world leader in GIS software                                                        1
RRP Brand Portfolio Executive at JTI (Japan Tobacco International)                                                       1
Information Syst

In [20]:
def clean_text(text):

  tokenizer = RegexpTokenizer(r'\w+')
  tokens = tokenizer.tokenize(text)
  lemmatizer = WordNetLemmatizer()
  tokens = [lemmatizer.lemmatize(token.lower(), pos='v') for token in tokens]
  keep_words = [token for token in tokens if token not in stopwords.words('english')]
  row_text = ' '.join(keep_words)
  row_text = ' '.join([word for word in row_text.split() if len(word)>1])  ## remove one letter words
  row_text = re.sub(r'\w*\d\w*', '', row_text).strip()
  return row_text

In [21]:
data['job_title'] = data['job_title'].apply(clean_text)

In [22]:
data.replace({'job_title' : {'magna cum laude' : 'with great distinction', 'chro' : 'chief human resources officer', 'svp' : 'senior vice president'
        ,'gphr' : 'global professional in human resources', 'hris' : 'human resources information system'
        , 'csr' : 'corporate social responsibility', 'sphr' : 'strategic and policy-making certification'
        , 'hr' : 'human resources', 'rrp' : 'recommended retail prices'}}, regex=True, inplace=True)

In [23]:
data['job_title'] = data['job_title'].apply(clean_text)

In [24]:
data.head()

,job_title,location,connection
id,,,
1,bauer college business graduate great distinct...,"Houston, Texas",85
2,native english teacher epik english program korea,Kanada,500+
3,aspire human resources professional,"Raleigh-Durham, North Carolina Area",44
4,people development coordinator ryan,"Denton, Texas",500+
5,advisory board member celal bayar university,"İzmir, Türkiye",500+


In [25]:
data.replace({'location' : {'Amerika Birleşik Devletleri' : 'United States of America', 'Houston, Texas Area' : 'Houston, Texas', 'Austin, Texas Area' : 'Austin, Texas',
                            'Kanada' : 'Canada', 'Kokomo, Indiana Area' : 'Kokomo, Indiana', 'Türkiye' : 'Turkey'}}, regex=True, inplace=True)

In [26]:
data.head()

,job_title,location,connection
id,,,
1,bauer college business graduate great distinct...,"Houston, Texas",85
2,native english teacher epik english program korea,Canada,500+
3,aspire human resources professional,"Raleigh-Durham, North Carolina Area",44
4,people development coordinator ryan,"Denton, Texas",500+
5,advisory board member celal bayar university,"İzmir, Turkey",500+


In [27]:
data['connection'].value_counts()

500+     18
4         2
7         2
57        2
40        1
18        1
349       1
155       1
39        1
85        1
50        1
9         1
415       1
19        1
71        1
48        1
103       1
64        1
174       1
268       1
455       1
52        1
409       1
212       1
16        1
5         1
82        1
390       1
2         1
61        1
1         1
44        1
49        1
Name: connection, dtype: int64

Word embedding techniques are used to represent words mathematically

- The higher the TF-IDF score the more important or relevant the term is; as a term gets less relevant, its TF-IDF score will approach 0.
- TF is the frequency of any 'term' in a given 'document'.
- IDF is constant per corpus, and accounts for the ratio of documents that include that specific 'term'.

TF-IDF vectorization involves calculating the TF-IDF score for every word in your corpus relative to that document and then putting that information into a vector. Thus each document in your corpus would have its own vector, and the vector would have a TF-IDF score for every single word in the entire collection of documents. Once you have these vectors you can apply them to various use cases such as seeing if two documents are similar by comparing their TF-IDF vector using cosine similarity.

Ref: https://www.capitalone.com/tech/machine-learning/understanding-tf-idf/

Symbolisation:
- t : term (word)
- d : document (set of words)
- D : corpus (the total document set)
- n : # of d in D 

--------------------------------------------------------------------------
- $tf(t,d)$ = count$(t : t \in d)$ / count(words $\in$ d)
 
 or
 
 $tf(t,d) = \mathrm{log}\bigl(count(t : t \in d) + 1 \bigr)$ 
--------------------------------------------------------------------------- 

- $idf(t,D) = \mathrm{log}\bigl(\frac{n}{count(d \in D: t \in d)}\bigr)$

 or from Skicit-learn

 $idf(t,D) = \mathrm{log}\bigl(\frac{n+1}{count(d \in D: t \in d)+1}\bigr)+1$ 

-------------------------------------------------------------------------

- $tfidf(t,d,D) = tf(t,d) \cdot idf(t,D)$, where $\cdot$ represents the dot product.

$\text{cosine similarity}=S_{C}(A,B):=\cos(\theta )={\mathbf {A} \cdot \mathbf {B}  \over \|\mathbf {A} \|\|\mathbf {B} \|}= \frac {\sum \limits _{i=1}^{n}A_{i}B_{i} } {\sqrt {\sum \limits _{i=1}^{n}A_{i}^{2} } \sqrt {\sum \limits _{i=1}^{n}B_{i}^{2} } }$

In [28]:
def tf_idf(feature, keyword,df):

    ## put df's rows in a list. This is our corpus.
    corpus = df[feature].tolist()

    ## append the keywords at the end of the corpus so we can compare it with the rest of the rows.
    corpus.append(clean_text(keyword))    
    
    ## create a matrix of tf-idf features.
    tfvec = TfidfVectorizer()    
    tf_idf_all = tfvec.fit_transform(corpus) ## vectorisation
    #tf_idf_all_df = pd.DataFrame(tf_idf_all.toarray(), columns = tfvec.get_feature_names())

    ## similarity of each row of the initial corpus with the keywords using cosine similarity.
    cos_sim = cosine_similarity(tf_idf_all.toarray()[:-1], tf_idf_all.toarray()[-1].reshape(1, -1))

    ## append the sorted similarities on a new column of our dataframe.
    df['cosine_similarity'] = cos_sim
    df = df.sort_values('cosine_similarity', ascending=False)
    return df

In [29]:
data_aspiring = data.copy()
data_aspiring = tf_idf('job_title','Aspiring human resources', data_aspiring)
data_aspiring.head(n=15)

,job_title,location,connection,cosine_similarity
id,,,,
97,aspire human resources professional,"Kokomo, Indiana",71,0.757836
3,aspire human resources professional,"Raleigh-Durham, North Carolina Area",44,0.757836
6,aspire human resources specialist,Greater New York City Area,1,0.665663
73,aspire human resources manager seek internship...,"Houston, Texas",7,0.601381
27,aspire human resources management student seek...,"Houston, Texas",500+,0.424475
72,business management major aspire human resourc...,"Monroe, Louisiana Area",5,0.422144
66,experience retail manager aspire human resourc...,"Austin, Texas",57,0.403974
74,human resources professional,Greater Boston Area,16,0.393191
7,student humber college aspire human resources ...,Canada,61,0.382504


In [30]:
data_seeking = data.copy()
data_seeking = tf_idf('job_title','seeking human resources', data_seeking)
data_seeking.head(n=10)

,job_title,location,connection,cosine_similarity
id,,,,
99,seek human resources position,"Las Vegas, Nevada Area",48,0.685990
28,seek human resources opportunities,"Chicago, Illinois",390,0.662609
73,aspire human resources manager seek internship...,"Houston, Texas",7,0.616508
10,seek human resources human resources informati...,Greater Philadelphia Area,500+,0.508548
27,aspire human resources management student seek...,"Houston, Texas",500+,0.449193
74,human resources professional,Greater Boston Area,16,0.372009
94,seek human resources opportunities open travel...,United States of America,415,0.359944
100,aspire human resources manager graduate may se...,"Cape Girardeau, Missouri",103,0.353925
75,nortia staff seek human resources payroll admi...,"San Jose, California",500+,0.320370


In [31]:
############word2vector

In [32]:
def word2vector(feature, keyword, df, min_count=1, window=5, epochs=10):

    ## put df's rows in a list. This is our corpus.
    corpus = df[feature].tolist()

    #kw = clean_text(keyword)
    
    tokens = [word_tokenize(row) for row in corpus]

    model = Word2Vec(tokens, min_count=min_count, window=window,sg=1,batch_words=10)
    model.train(tokens, total_examples=1, epochs=epochs)

    p = np.array(0.)
    for k in keyword:
      p = p + model.wv[k]
    
    cs = []
    
    for i in tokens:
      a = np.array(0.)
      for j in i:
        if j in keyword:
          a = a + model.wv[j]
      cs.append(cosine_similarity(np.mean(np.array(p)).reshape(-1, 1), np.mean(a).reshape(-1, 1)).item())
    
    #cs_series = pd.Series(cs)
    df['cosine_similarity'] = np.array(cs)
    df = df.sort_values('cosine_similarity', ascending=False)
    return df

In [33]:
d_w2v = data.copy()
d_w2v = word2vector('job_title', ['aspire', 'human', 'resources'], d_w2v)
d_w2v

,job_title,location,connection,cosine_similarity
id,,,,
1,bauer college business graduate great distinct...,"Houston, Texas",85,1.0
83,human resources manager endemol shine north am...,"Los Angeles, California",268,1.0
74,human resources professional,Greater Boston Area,16,1.0
75,nortia staff seek human resources payroll admi...,"San Jose, California",500+,1.0
76,aspire human resources professional passionate...,"New York, New York",212,1.0
77,human resources conflict management policies p...,Dallas/Fort Worth Area,409,1.0
79,liberal arts major aspire human resources analyst,"Baton Rouge, Louisiana Area",7,1.0
81,senior human resources business partner heil e...,"Chattanooga, Tennessee Area",455,1.0
82,aspire human resources professional energetic ...,"Austin, Texas",174,1.0


In [34]:
###########fasttext

In [35]:
def fast_text(feature, keyword, df):

    ## put df's rows in a list. This is our corpus.
    corpus = df[feature].tolist()

    tokens = [word_tokenize(row) for row in corpus]

    model = FastText(tokens,sg=1,batch_words=10,hs=1)
   
    cs = []
    for i in tokens:
      cs.append(model.wv.n_similarity(i, keyword))

    df['similarity'] = np.array(cs)
    df = df.sort_values('similarity', ascending=False)
    return df

In [36]:
d_fasttext = data.copy()
d_fasttext = fast_text('job_title', ['aspire', 'human', 'resources'], d_fasttext)
d_fasttext

,job_title,location,connection,similarity
id,,,,
3,aspire human resources professional,"Raleigh-Durham, North Carolina Area",44,0.938595
97,aspire human resources professional,"Kokomo, Indiana",71,0.938595
6,aspire human resources specialist,Greater New York City Area,1,0.925602
73,aspire human resources manager seek internship...,"Houston, Texas",7,0.842769
74,human resources professional,Greater Boston Area,16,0.748578
66,experience retail manager aspire human resourc...,"Austin, Texas",57,0.739099
27,aspire human resources management student seek...,"Houston, Texas",500+,0.720242
7,student humber college aspire human resources ...,Canada,61,0.695528
68,human resources specialist luxottica,Greater New York City Area,500+,0.667126


In [37]:
# c = data['job_title'].tolist()
# t = [word_tokenize(row) for row in c]
# m = FastText(t,sg=1,batch_words=10,hs=1)
# corpus = Corpus() 
# #training the corpus to generate the co occurence matrix which is used in GloVe
# corpus.fit(t, window=10)
# #creating a Glove object which will use the matrix created in the above lines to create embeddings
# #We can set the learning rate as it uses Gradient Descent and number of components
# g = Glove(no_components=10, learning_rate=0.05)
# g.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
# g.add_dictionary(corpus.dictionary)


In [38]:
########GLOVE

In [39]:
def glove_(feature, keyword, df):

    # creating a corpus object
    corpus = Corpus() 
    tokens = [word_tokenize(row) for row in df[feature].tolist()]
    #training the corpus to generate the co occurence matrix which is used in GloVe
    corpus.fit(tokens, window=10)
    #creating a Glove object which will use the matrix created in the above lines to create embeddings
    #We can set the learning rate as it uses Gradient Descent and number of components
    glove = Glove(no_components=10, learning_rate=0.05)
    
    glove.fit(corpus.matrix, epochs=30, no_threads=4, verbose=True)
    glove.add_dictionary(corpus.dictionary)


    p = np.array(0.)
    for k in keyword:
      p = p + glove.word_vectors[glove.dictionary[k]]
    
    cs = []
    
    for i in tokens:
      a = np.array(0.)
      for j in i:
        if j in keyword:
          a = a + glove.word_vectors[glove.dictionary[j]]
      cs.append(cosine_similarity(np.mean(np.array(p)).reshape(-1, 1), np.mean(a).reshape(-1, 1)).item())
    
    #cs_series = pd.Series(cs)
    df['cosine_similarity'] = np.array(cs)
    df = df.sort_values('cosine_similarity', ascending=False)
    return df

In [40]:
d_glove = data.copy()
d_glove = glove_('job_title', ['aspire', 'human', 'resources'], d_glove)
d_glove

Performing 30 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20
Epoch 21
Epoch 22
Epoch 23
Epoch 24
Epoch 25
Epoch 26
Epoch 27
Epoch 28
Epoch 29


,job_title,location,connection,cosine_similarity
id,,,,
1,bauer college business graduate great distinct...,"Houston, Texas",85,1.0
83,human resources manager endemol shine north am...,"Los Angeles, California",268,1.0
74,human resources professional,Greater Boston Area,16,1.0
75,nortia staff seek human resources payroll admi...,"San Jose, California",500+,1.0
76,aspire human resources professional passionate...,"New York, New York",212,1.0
77,human resources conflict management policies p...,Dallas/Fort Worth Area,409,1.0
79,liberal arts major aspire human resources analyst,"Baton Rouge, Louisiana Area",7,1.0
81,senior human resources business partner heil e...,"Chattanooga, Tennessee Area",455,1.0
82,aspire human resources professional energetic ...,"Austin, Texas",174,1.0


predicting neighboring words (CBOW) or predicting the focus word (Skip-Gram)

The main principle behind fastText is that the morphological structure of a word carries important information about the meaning of the word. Such structure is not taken into account by traditional word embeddings like Word2Vec, which train a unique word embedding for every individual word.

TF-IDF does not take into account the semantic meaning or context of the words whereas BERT does